In [218]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [219]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [220]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [221]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [245]:
sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

def seed_np(integer):
    np.random.seed(integer)

class ERegressor:
    class Net:
        def __init__(self):
            pass

        def set_input_size(self, input_size):
            self.input_size = input_size
        
        def set_weights(self, weights_and_biases):
            self.w1 = weights_and_biases[0]
            self.w2 = weights_and_biases[1]
            self.b1 = weights_and_biases[2]
            self.b2 = weights_and_biases[3]
        
        def set_mutation_sigma(self, n):
            #self.mutation_sigma = 3 - (2 - 40 / (10 + n))
            self.mutation_sigma = 2.5 - (3 - 1200 / (200 + n))
        
        def predict(self, input):
            return (self.w2.T @ sigmoid(self.w1.T @ input.T + self.b1) + self.b2).reshape(-1)
        
        def __add__(self, other):
            w1 = (self.w1 + other.w1) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, self.input_size))
            w2 = (self.w2 + other.w2) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, 1))
            b1 = (self.b1 + other.b1) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, 1))
            b2 = (self.b2 + other.b2) / 2 + np.random.normal(0, self.mutation_sigma, 1)

            return w1, w2, b1, b2
    
    def __init__(self, n = 100, random_state = None):
        self.nets = {}
        self.y_preds = {}
        self.best_net = None
        self.best_result = None
        self.n = n // 2 * 2
        
        for i in range(self.n):
            self.nets[i] = self.Net()

        if random_state != None:
            seed_np(random_state)
    
    def fit(self, X_train, y_train, epochs = 100):
        self.input_size = X_train.shape[1]

        for key in self.nets.keys():
            self.nets[key].set_input_size(self.input_size)

            self.nets[key].set_weights((
                np.random.uniform(-3, 3, (self.input_size, self.input_size)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, 1))
            )

        for epoch in range(epochs):
            for key, _ in self.nets.items():
                self.y_preds[key] = self.nets[key].predict(X_train)
                self.nets[key].set_mutation_sigma(epoch)
            
            self.mean_absolute_errors = {}
            
            for key, _ in self.y_preds.items():
                self.mean_absolute_errors[key] = mean_absolute_error(y_train, self.y_preds[key])

            self.sorted_indecies = [key for key, _ in sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])]

            if self.best_result != None:
                if self.best_result[1] > sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0][1]:
                    self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                    print(f"Epoch {epoch}: MAE: {self.best_result[1]}")
            else:        
                self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                print(f"Epoch {epoch}: MAE: {self.best_result[1]}")

            self.best_net = self.sorted_indecies[0]
            
            for i in range(0, self.n // 2, 2):
                self.nets[self.sorted_indecies[self.n // 2 + i]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                self.nets[self.sorted_indecies[self.n // 2 + i + 1]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
            

    def predict(self, X):
        return self.nets[self.best_net].predict(X)

In [234]:
for n in range(0, 1000, 10):
    #print(2.3 - (3 - 1200 / (200 + n)), end = " ")
    #print(3 - (2 - 40 / (10 + n)), end = " ")
    pass

In [246]:
regressor = ERegressor(n = 100, random_state = 42)
regressor.fit(scaled_X_train, y_train, epochs = 1000)

Epoch 0: MAE: 15.525692036080034
Epoch 1: MAE: 7.477269631428442
Epoch 3: MAE: 5.249829589371986
Epoch 6: MAE: 4.6856046881029165
Epoch 17: MAE: 4.475585814564628
Epoch 24: MAE: 4.274454356990163
Epoch 45: MAE: 4.111198203965975
Epoch 51: MAE: 4.045583500255095
Epoch 52: MAE: 3.9238223486310075
Epoch 58: MAE: 3.801227160802128
Epoch 72: MAE: 3.68821181874302
Epoch 86: MAE: 3.146313691916892
Epoch 89: MAE: 3.1189069844357347
Epoch 115: MAE: 3.0546884078634
Epoch 148: MAE: 2.9100331780460524
Epoch 194: MAE: 2.8836876561069924
Epoch 198: MAE: 2.871253771610806
Epoch 231: MAE: 2.7286360335363438
Epoch 299: MAE: 2.7164837825762493
Epoch 314: MAE: 2.71295922869991
Epoch 319: MAE: 2.6652028657813918
Epoch 323: MAE: 2.6526363486135773
Epoch 340: MAE: 2.611507715892798
Epoch 351: MAE: 2.60034217902979
Epoch 370: MAE: 2.5395492214974973
Epoch 400: MAE: 2.516854430144453
Epoch 404: MAE: 2.4778764598827534
Epoch 409: MAE: 2.4737276647526203
Epoch 419: MAE: 2.3915014564571773
Epoch 454: MAE: 2.3806

In [247]:
y_pred = regressor.predict(scaled_X_test)

In [248]:
mean_absolute_error(y_test, y_pred)

2.066978884294728

In [244]:
regressor.nets[81].w1

array([[ -2.30617   ,   4.29951232,  -6.99754506,  -4.40570208,
         -0.5850187 ,   8.87132895,  -0.42133986,  -3.02297951,
          5.16772472],
       [  4.448949  ,  -3.95192358, -10.2689638 , -12.92778308,
         -0.32907322,   2.87757324,   3.3103889 ,  -1.90412917,
          1.91461245],
       [  3.07288752, -23.95703469,   0.99954109,  -5.96031325,
         -4.34716271,   1.84886888,   5.06499244,   3.54435692,
        -14.02571242],
       [-10.99604624,  -1.87087873, -10.86252443,  -0.77844065,
         11.77868102,   7.21721941,  -1.42865877,   2.20863205,
        -10.89086662],
       [ -4.23354936,   0.72116499,  -4.94715098,   7.01261169,
        -10.78468281,  -1.39009469, -17.17951638, -12.09440963,
          7.41085784],
       [ -3.12143971,   2.39661362, -12.41584429,  -6.92141497,
         -2.77019087,  -2.20527365,  -0.70303687,  14.77331331,
         -5.74384873],
       [  1.93954017,   2.93210171,  -8.55972049,  -8.5103172 ,
          1.96237237,   2.0827